### Practical 6

### Read two set of documents, perform tokeization, map dictionaries, create corpus and calculate similarity between the documents

In [1]:
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
import numpy as np

In [2]:
text = "The tiger is the largest living cat species and a member of the genus Panthera. It is most recognisable for its dark vertical stripes on orange-brown fur with a lighter underside. It is an apex predator, primarily preying on ungulates such as deer and wild boar."

In [3]:
sent_tokens = sent_tokenize(text)
print(sent_tokens)

['The tiger is the largest living cat species and a member of the genus Panthera.', 'It is most recognisable for its dark vertical stripes on orange-brown fur with a lighter underside.', 'It is an apex predator, primarily preying on ungulates such as deer and wild boar.']


In [5]:
word_tokens = word_tokenize(text)
print(word_tokens)

['The', 'tiger', 'is', 'the', 'largest', 'living', 'cat', 'species', 'and', 'a', 'member', 'of', 'the', 'genus', 'Panthera', '.', 'It', 'is', 'most', 'recognisable', 'for', 'its', 'dark', 'vertical', 'stripes', 'on', 'orange-brown', 'fur', 'with', 'a', 'lighter', 'underside', '.', 'It', 'is', 'an', 'apex', 'predator', ',', 'primarily', 'preying', 'on', 'ungulates', 'such', 'as', 'deer', 'and', 'wild', 'boar', '.']


In [28]:
doc = ""
length = 0
with open('file1.txt', 'r') as f:
    doc = f.read()
print(doc)
sent_tokens = sent_tokenize(doc)
print('Number of lines = ', len(sent_tokens))

Mars is the fourth planet from the sun. It is the second smallest planet in the solar system after mercury. Saturn is yellow planet.
Number of lines =  3


In [29]:
# Tokenize words:
word_tokens = [[w.lower() for w in word_tokenize(text)] for text in sent_tokens]
print(word_tokens)

[['mars', 'is', 'the', 'fourth', 'planet', 'from', 'the', 'sun', '.'], ['it', 'is', 'the', 'second', 'smallest', 'planet', 'in', 'the', 'solar', 'system', 'after', 'mercury', '.'], ['saturn', 'is', 'yellow', 'planet', '.']]


In [30]:
# Create the dictionary using gensim
dictionary = gensim.corpora.Dictionary(word_tokens)
print(dictionary.token2id)

{'.': 0, 'fourth': 1, 'from': 2, 'is': 3, 'mars': 4, 'planet': 5, 'sun': 6, 'the': 7, 'after': 8, 'in': 9, 'it': 10, 'mercury': 11, 'second': 12, 'smallest': 13, 'solar': 14, 'system': 15, 'saturn': 16, 'yellow': 17}


In [31]:
# Create a corpus/vectorization:
corpus = [dictionary.doc2bow(token) for token in word_tokens]
print(corpus) # will give (vector_number_given_in_the_step_above, frequency_of_that_word_in_the_sentence) format

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2)], [(0, 1), (3, 1), (5, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)], [(0, 1), (3, 1), (5, 1), (16, 1), (17, 1)]]


In [40]:
# Calculate the frequency (TFID Term frequency)
tf_idf = gensim.models.TfidfModel(corpus)
#print(tf_idf[corpus], '\n')
for doc in tf_idf[corpus]:
    #print(doc, '\n')
    print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc], '\n')

[['fourth', 0.47], ['from', 0.47], ['mars', 0.47], ['sun', 0.47], ['the', 0.35]] 

[['the', 0.25], ['after', 0.34], ['in', 0.34], ['it', 0.34], ['mercury', 0.34], ['second', 0.34], ['smallest', 0.34], ['solar', 0.34], ['system', 0.34]] 

[['saturn', 0.71], ['yellow', 0.71]] 



In [42]:
# Similarity measure
sims = gensim.similarities.Similarity('', tf_idf[corpus], num_features=len(dictionary)) # '' will have the working directory

In [44]:
query_doc = ""
with open('file2.txt', 'r') as f1:
    query_doc = f1.read()
print(query_doc)
q_sent_tokens = sent_tokenize(query_doc)
print('Number of lines = ', len(q_sent_tokens))

Saturn is sixth planet from Sun.
Number of lines =  1


In [48]:
# Word tokenize it and convert into bag of words
for line in q_sent_tokens:
    q_word_tokens = [w.lower() for w in word_tokenize(line)]
    q_bow = dictionary.doc2bow(q_word_tokens)
print(q_word_tokens)
print(q_bow)

['saturn', 'is', 'sixth', 'planet', 'from', 'sun', '.']
[(0, 1), (2, 1), (3, 1), (5, 1), (6, 1), (16, 1)]


In [49]:
# Perform a similarity query against the original corpus:
query_tf_idf = tf_idf[q_bow]
print(query_tf_idf)
print('Comparing results:', sims[query_tf_idf])

[(2, 0.5773502691896258), (6, 0.5773502691896258), (16, 0.5773502691896258)]
Comparing results: [0.5416385  0.         0.40824828]


In [50]:
# Adding the similarities
import numpy as np
sum_of_sims =(np.sum(sims[query_tf_idf], dtype=np.float32))
print(sum_of_sims)

0.9498868


In [51]:
# Getting the similarity percentage:
percentage_of_similarity = round(float((sum_of_sims / len(word_tokens)) * 100))
print(f'Average similarity float: {float(sum_of_sims / len(word_tokens))}')
print(f'Average similarity percentage: {float(sum_of_sims / len(word_tokens)) * 100}')
print(f'Average similarity rounded percentage: {percentage_of_similarity}')

Average similarity float: 0.31662893295288086
Average similarity percentage: 31.662893295288086
Average similarity rounded percentage: 32
